In [1]:
# clone PaddleOCR环境 和 PaddleSlim环境
!git clone https://gitee.com/paddlepaddle/PaddleOCR -b dygraph
!git clone https://gitee.com/paddlepaddle/PaddleSlim -b release/2.0.0

fatal: destination path 'PaddleOCR' already exists and is not an empty directory.
fatal: destination path 'PaddleSlim' already exists and is not an empty directory.


In [3]:
# 复制修改文件到对应文件夹
!cp /home/aistudio/work/ppocr/modeling/architectures/plugnet.py /home/aistudio/PaddleOCR/ppocr/modeling/architectures/plugnet.py

!cp /home/aistudio/work/ppocr/modeling/necks/rnn.py /home/aistudio/PaddleOCR/ppocr/modeling/necks/rnn.py
!cp /home/aistudio/work/ppocr/modeling/necks/rcan.py /home/aistudio/PaddleOCR/ppocr/modeling/necks/rcan.py
!cp /home/aistudio/work/ppocr/modeling/necks/__init__.py /home/aistudio/PaddleOCR/ppocr/modeling/necks/__init__.py

!cp /home/aistudio/work/ppocr/modeling/heads/rec_att_head.py /home/aistudio/PaddleOCR/ppocr/modeling/heads/rec_att_head.py

!cp /home/aistudio/work/ppocr/modeling/backbones/__init__.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/__init__.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_resnet_vd_se.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_resnet_vd_se.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_rexnet.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_rexnet.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_resnet_metaacon.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_resnet_metaacon.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_resnet_se_metaacon.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_resnet_se_metaacon.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_resnet_vd_se_fpna.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_resnet_vd_se_fpna.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_rednet_vd.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_rednet_vd.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_mobilenext.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_mobilenext.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_mobilenet_v3_plug.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_mobilenet_v3_plug.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_mobilenet_v3_ca.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_mobilenet_v3_ca.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_mobilenet_v3_acon.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_mobilenet_v3_acon.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_mobilenet_v3_fpn.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_mobilenet_v3_fpn.py
!cp /home/aistudio/work/ppocr/modeling/backbones/rec_mobilenet_v3_fpn_blurpool.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/rec_mobilenet_v3_fpn_blurpool.py
!cp /home/aistudio/work/ppocr/modeling/backbones/blurpool.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/blurpool.py
!cp /home/aistudio/work/ppocr/modeling/backbones/acon.py /home/aistudio/PaddleOCR/ppocr/modeling/backbones/acon.py

!cp /home/aistudio/work/ppocr/losses/rec_plug_loss.py /home/aistudio/PaddleOCR/ppocr/losses/rec_plug_loss.py
!cp /home/aistudio/work/ppocr/losses/__init__.py /home/aistudio/PaddleOCR/ppocr/losses/__init__.py

!cp /home/aistudio/work/ppocr/data/imaug/rec_img_aug.py /home/aistudio/PaddleOCR/ppocr/data/imaug/rec_img_aug.py

!cp /home/aistudio/work/tools/egaleeye_prune.py /home/aistudio/PaddleOCR/tools/egaleeye_prune.py
!cp /home/aistudio/work/tools/export_pruned_model.py /home/aistudio/PaddleOCR/tools/export_pruned_model.py
!cp /home/aistudio/work/tools/export_model.py /home/aistudio/PaddleOCR/tools/export_model.py
!cp /home/aistudio/work/tools/infer_rec.py /home/aistudio/PaddleOCR/tools/infer_rec.py
!cp /home/aistudio/work/tools/infer/predict_rec.py /home/aistudio/PaddleOCR/tools/infer/predict_rec.py
!cp /home/aistudio/work/tools/infer/utility.py /home/aistudio/PaddleOCR/tools/infer/utility.py
!cp /home/aistudio/work/tools/model_summary.py /home/aistudio/PaddleOCR/tools/model_summary.py
!cp /home/aistudio/work/tools/program.py /home/aistudio/PaddleOCR/tools/program.py
!cp /home/aistudio/work/tools/prune.py /home/aistudio/PaddleOCR/tools/prune.py
!cp /home/aistudio/work/tools/train.py /home/aistudio/PaddleOCR/tools/train.py


In [3]:
# 安装PaddleOCR环境 和 PaddleSlim环境
!cd PaddleOCR && pip install -r requirements.txt
#!cd PaddleOCR && python setup.py install

!cd PaddleSlim && pip install -r requirements.txt
#!cd PaddleSlim && python setup.py install

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.0MB 15.5MB/s eta 0:00:01
     |████████████████████████████████| 12.5MB 8.7MB/s eta 0:00:011
     |████████████████████████████████| 952kB 20.4MB/s eta 0:00:01
     |████████████████████████████████| 133kB 64.1MB/s eta 0:00:01
     |████████████████████████████████| 307kB 22.3MB/s eta 0:00:01
     |████████████████████████████████| 51kB 15.4MB/s eta 0:00:01
     |████████████████████████████████| 34.2MB 8.4MB/s eta 0:00:011
     |████████████████████████████████| 174kB 24.6MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 30.0MB/s eta 0:00:01


In [4]:
# 解压数据集
!unzip /home/aistudio/data/data87683/训练数据集.zip -d /home/aistudio/data/train_images
!unzip /home/aistudio/data/data87685/A榜测试数据集.zip -d /home/aistudio/data/test_images

In [5]:
# 预处理数据集标签
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
def Q2B(s):
    """全角转半角"""
    inside_code=ord(s)
    if inside_code==0x3000:
        inside_code=0x0020
    else:
        inside_code-=0xfee0
    if inside_code<0x0020 or inside_code>0x7e: #转完之后不是半角字符返回原来的字符
        return s
    return chr(inside_code)

def stringQ2B(s):
    """把字符串全角转半角"""
    return "".join([Q2B(c) for c in s])

def is_chinese(s):
    """判断unicode是否是汉字"""
    for c in s:
        if c < u'\u4e00' or c > u'\u9fa5':
            return False
    return True

def is_number(s):
    """判断unicode是否是数字"""
    for c in s:
        if c < u'\u0030' or c > u'\u0039':
            return False
    return True

def is_alphabet(s):
    """判断unicode是否是英文字母"""
    for c in s:
        if (u'\u0041'<= c <= u'\u005a') or (u'\u0061'<= c <= u'\u007a'):
            return True
    return False

def del_other(s):
    """判断是否非汉字，数字和英文"""
    res = str()
    for c in s:
        if not (is_chinese(c) or is_number(c) or is_alphabet(c)):
            c = ""
        res += c
    return res

# 原始label文件
src_label_file = "/home/aistudio/data/train_images/训练数据集/LabelTrain.txt"
dst_label_file = "/home/aistudio/work/label.txt"
label_list_file = "/home/aistudio/work/label_dict.txt"
src_data_root = "/home/aistudio/data/train_images/训练数据集/TrainImages"

m_width = 0
m_height = 0
label_max_len = 0
width_dict = dict()
height_dict = dict()
widths = list()
heights = list()
classSet = set()

with open(src_label_file,'r', encoding= 'utf-8') as src_label, \
     open(dst_label_file, "w", encoding= 'utf-8') as dst_label:

    lines = src_label.readlines()

    for line in lines:
        path, label = line.split("\t")
        img = Image.open(os.path.join(src_data_root, path))
        width, height = img.size
        
        # 全角转半角
        label = stringQ2B(label.split('\n')[0])
        # # 删除特殊符号
        # label = del_other(label)

        # 记录数据集中最长标签
        if len(label) > label_max_len:
            label_max_len = len(label)

        if label != "":
            dst_label.write(path + "\t" + label + "\n")
        
        for e in label:
            classSet.add(e)
            
        widths.append(width)
        heights.append(height)
        width_dict[int(width)] = 1 if (int(width)) not in width_dict else 1 + width_dict[int(width)]
        height_dict[int(height)] = 1 if (int(height)) not in height_dict else 1 + height_dict[int(height)]
        m_width += width
        m_height += height

m_width = m_width/len(lines)
m_height = m_height/len(lines)

aspect_ratio = m_width/m_height

width_dict = dict(sorted(width_dict.items(), key=lambda item: item[1], reverse=True))
height_dict = dict(sorted(height_dict.items(), key=lambda item: item[1], reverse=True))

print("aspect ratio is: {}, mean width is: {}, mean height is: {}".format(aspect_ratio,m_width,m_height))
print("Width dict:{}".format(width_dict))
print("Height dict:{}".format(height_dict))
print("Label max len:{}".format(label_max_len))

plt.hist(widths,bins = len(width_dict))
plt.show()
plt.hist(heights,bins = len(height_dict))
plt.show()

with open(label_list_file, "w", encoding= 'utf-8') as label_list:
    classList = sorted(list(classSet))
    for idx, c in enumerate(classList):
        label_list.write("{}\n".format(c))

print("classify num: ", len(classSet))
print("Train label save in {}".format(dst_label_file))
print("Train list save in {}".format(label_list_file))

In [6]:
import random
import os
import shutil
# 按8：2将数据集划分为训练集和验证集
ratio = 0.8
seed = 12

dst_train_root = "/home/aistudio/data/rec/train"
dst_sample_train_root = "/home/aistudio/data/rec/sample_train"
dst_eval_root = "/home/aistudio/data/rec/eval"

if not os.path.exists(dst_train_root):
    os.makedirs(dst_train_root)

if not os.path.exists(dst_sample_train_root):
    os.makedirs(dst_sample_train_root)

if not os.path.exists(dst_eval_root):
    os.makedirs(dst_eval_root)

print(dst_label_file)
dst_train_label_file = "/home/aistudio/data/rec/rec_gt_train.txt"
dst_sample_train_label_file = "/home/aistudio/data/rec/rec_gt_sample_train.txt"
dst_eval_label_file = "/home/aistudio/data/rec/rec_gt_eval.txt"

with open(dst_label_file, "r", encoding= 'utf-8') as label_file:
    lines = label_file.readlines()
    random.seed(seed)
    random.shuffle(lines)
    train_lines = lines[0:int(ratio*len(lines))]
    sample_train_lines = lines[0:int((1-ratio)*len(lines))+1]
    eval_lines = lines[int(ratio*len(lines)):len(lines)]
    
    # 创建训练集
    for idx, line in enumerate(train_lines):
        path, label = line.split("\t")
        shutil.copy(os.path.join(src_data_root,path), os.path.join(dst_train_root,path))
        train_lines[idx] = line.replace("img", "rec/train/img")
    
    # 创建部分训练集
    for idx, line in enumerate(sample_train_lines):
        path, label = line.split("\t")
        shutil.copy(os.path.join(src_data_root,path), os.path.join(dst_sample_train_root,path))
        sample_train_lines[idx] = line.replace("img", "rec/sample_train/img")

    # 创建验证集
    for idx, line in enumerate(eval_lines):
        path, label = line.split("\t")
        shutil.copy(os.path.join(src_data_root,path), os.path.join(dst_eval_root,path))
        eval_lines[idx] = line.replace("img", "rec/eval/img")


with open(dst_train_label_file, "w", encoding= 'utf-8') as train_label_file, \
     open(dst_sample_train_label_file, "w", encoding= 'utf-8') as sample_train_label_file, \
     open(dst_eval_label_file, "w", encoding= 'utf-8') as eval_label_file:
    train_label_file.writelines(train_lines)
    sample_train_label_file.writelines(sample_train_lines)
    eval_label_file.writelines(eval_lines)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 